In [ ]:
import time
import hashlib
import requests
import hmac
from urllib.parse import urlencode
import datetime
import os

S_URL_V1 = "https://api.binance.com/sapi/v1"

api_key = ''
secret_key = ''

def _sign(params={}):
    data = params.copy()
    ts = str(int(1000 * time.time()))
    data.update({"timestamp": ts})     
    h = urlencode(data)
    hh = h.replace("%40", "@")
    print(hh)
    b = bytearray()
    b.extend(secret_key.encode())
    signature = hmac.new(b, msg=h.encode('utf-8'), digestmod=hashlib.sha256).hexdigest()
    sig = {"signature": signature}
    print(signature)
    return data, sig

def post(path, params={}):
    sign = _sign(params)
    query = urlencode(sign[0]) + "&" + urlencode(sign[1])
    url = "%s?%s" % (path, query)
    print(url)
    header = {"X-MBX-APIKEY": api_key}
    print(header)
    p = requests.post(url, headers=header, \
        timeout=30, verify=True)
    return p

def get(path, params):
    sign = _sign(params)
    query = urlencode(sign[0]) + "&" + urlencode(sign[1])
    url = "%s?%s" % (path, query)
    print(url)
    header = {"X-MBX-APIKEY": api_key}
    p = requests.get(url, headers=header, \
        timeout=30, verify=True)
    return p

def get_ms_from_str(date_time, str_format='%Y%m%d %H:%M:%S'):
    # print(int(datetime.datetime.now().timestamp() * 1000))
    return int(datetime.datetime.strptime(date_time, str_format).timestamp() * 1000 + 3.6e6)

In [ ]:
print(get_ms_from_str('20200101 00:00:00'))
print(1577836800000)

In [ ]:
start_time = get_ms_from_str('20210102 00:00:00')
endt_time = get_ms_from_str('20210103 23:59:59')

path = "%s/futuresHistDataId" % S_URL_V1   
params = {"symbol": 'BTCUSDT',
          "startTime": start_time,
          "endTime": endt_time,
          "dataType": 'S_DEPTHS',
         }

# T_TRADE  tick-by-tick trade
# T_DEPTH  tick-by-tick order book (level 2); under development.
# S_DEPTH  order book snapshot (level 2); temp data solution only for BTCUSDT at the moment.

result = post(path, params)
downloadId = result.json()['id']

# write ids to file
ofile = 'id_log.txt'
write_header = False
if not os.path.exists(ofile) or os.path.getsize(ofile) == 0:
    write_header = True

with open(ofile, 'a') as out:
    if write_header:
        out.write('log_time,symbol,startTime,endTime,dataType,id\n')
    out.write(datetime.datetime.today().strftime('%Y%m%d_%H%M%S')
              + ',' + params['symbol']
              + ',' + str(params['startTime'])
              + ',' + str(params['endTime'])
              + ',' + params['dataType']
              + ',' + str(downloadId)
              + '\n')

In [ ]:
print(result.json())

In [ ]:
path = "%s/downloadLink" % S_URL_V1   
params = {"downloadId": 32869}
result = get(path, params)

print()
print(result.json())
print('now = ', int(datetime.datetime.now().timestamp()))

In [ ]:
result.json()['link']